In [1]:
import tensorflow as tf
import logging
import pickle
import random
import os
from tensorflow.contrib import rnn
from rouge import rouge


os.environ["CUDA_VISIBLE_DEVICES"] = "4"
os.environ["CUDA_VISIBLE_DEVICES"] = "5"
os.environ["CUDA_VISIBLE_DEVICES"] = "6"
os.environ["CUDA_VISIBLE_DEVICES"] = "7"

/data1/songbohan/anaconda3/envs/reinforce_learning/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [ ]:
# add train log module
log_fn = "./log/train.20181022.log"
logger = logging.getLogger("training")
hdlr = logging.FileHandler(log_fn)
logger.addHandler(hdlr)
logger.setLevel(logging.INFO)
tensorboard_log_path = "./log/"
model_dump_dir="pointer"
# model_load_dir="../model/20180918"
inference_fn = "./log/pointer.samples"

In [ ]:
# basic_data_

data_fn = "../data/finance150.batch.pkl"
with open(data_fn, 'rb') as f:
    abstract_train, abstract_test, title_train, title_test = pickle.load(open(data_fn, 'rb'))

In [ ]:
# hyper parameter

beg, eos, emp, unk = 0, 1, 2, 3
learning_rate = 0.001

save_epoc_step = 2
dropout_keep_prob = 1.
restore = False
inference = False
train = True
batch_size = 128
epocs = 1500

maxlena = 200
maxlent = 20
maxlen = maxlena + maxlent
maxlenh = maxlent
maxlend = maxlena

vocab_size = 80000
# idx2word = {idx:word for word, idx in word2idx.items()}
embedding_size = 100
memory_dim = 512

# cnn A, C
filter_sizes = [3,3,3,3,3]
num_filters = 1




In [ ]:
class Vocab(object):
    """Vocabulary class for mapping between words and ids (integers)"""

    def __init__(self, vocab_file, max_size):
        """Creates a vocab of up to max_size words, reading from the vocab_file. If max_size is 0, reads the entire vocab file.

        Args:
          vocab_file: path to the vocab file, which is assumed to contain "<word> <frequency>" on each line, sorted with most frequent word first. This code doesn't actually use the frequencies, though.
          max_size: integer. The maximum size of the resulting Vocabulary."""
        self._word_to_id = {}
        self._id_to_word = {}
        self._count = 0  # keeps track of total number of words in the Vocab

        # [UNK], [PAD], [START] and [STOP] get the ids 0,1,2,3.
        self._word_to_id["_unk"] = unk
        self._word_to_id["_beg"] = beg
        self._word_to_id["_eos"] = eos
        self._word_to_id["_emp"] = emp
        

        # Read the vocab file and add words up to max_size
        with open(vocab_file, 'r', encoding="utf-8") as vocab_f:
            for line in vocab_f:
                w = line.strip().lower()
                if w in self._word_to_id:
                    print("Duplicate:", w)
                    continue
                self._word_to_id[w] = self._count
                self._id_to_word[self._count] = w
                self._count += 1
                if max_size != 0 and self._count >= max_size:
                    print("max_size of vocab was specified as %i; we now have %i words. Stopping reading." % (
                    max_size, self._count))
                    break

        print("Finished constructing vocabulary of %i total words. Last ten word added: %s" % (
        self._count, " ".join([self._id_to_word[self._count - i - 1] for i in range(10)])))

    def word2id(self, word):
        """Returns the id (integer) of a word (string). Returns [UNK] id if word is OOV."""
        if word not in self._word_to_id:
            return self._word_to_id["_unk"]
        return self._word_to_id[word.lower()]

    def id2word(self, word_id):
        """Returns the word (string) corresponding to an id (integer)."""
        if word_id not in self._id_to_word:
            raise ValueError('Id not found in vocab: %d' % word_id)
        return self._id_to_word[word_id]

    def size(self):
        """Returns the total size of the vocabulary"""
        return self._count

#     def write_metadata(self, fpath):
#         """Writes metadata file for Tensorboard word embedding visualizer as described here:
#           https://www.tensorflow.org/get_started/embedding_viz

#         Args:
#           fpath: place to write the metadata file
#         """
#         print("Writing word embedding metadata file to %s..." % (fpath))
#         with open(fpath, "wb") as f:
#             fieldnames = ['word']
#             writer = csv.DictWriter(f, delimiter="\t", fieldnames=fieldnames)
#             for i in range(self.size()):
#                 writer.writerow({"word": self._id_to_word[i].encode("utf-8")})

#     def LoadWordEmbedding(self, w2v_file, word_dim):
#         self.wordDict = {}
#         self.word_dim = word_dim

#         self.wordDict[UNKNOWN_TOKEN] = np.zeros(self.word_dim, dtype=np.float32)
#         self.wordDict[PAD_TOKEN] = np.random.uniform(-1, 1, self.word_dim)
#         self.wordDict[START_DECODING] = np.random.uniform(-1, 1, self.word_dim)
#         self.wordDict[STOP_DECODING] = np.random.uniform(-1, 1, self.word_dim)
#         with open(w2v_file) as wf:
#             for line in wf:
#                 info = line.strip().split()
#                 word = info[0]
#                 coef = np.asarray(info[1:], dtype='float32')
#                 self.wordDict[word] = coef
#                 assert self.word_dim == len(coef)

#         self.MakeWordEmbedding()

#     def MakeWordEmbedding(self):
#         sorted_x = sorted(self._word_to_id.items(), key=operator.itemgetter(1))
#         self._wordEmbedding = np.zeros((self.size(), self.word_dim),
#                                        dtype=np.float32)  # replace unknown words with UNKNOWN_TOKEN embedding (zero vector)
#         for word, i in sorted_x:
#             if word in self.wordDict:
#                 self._wordEmbedding[i, :] = self.wordDict[word.lower()]
#         print('Word Embedding Reading done.')

#     def getWordEmbedding(self):
#         return self._wordEmbedding

In [ ]:
vocab = Vocab("../data/finance150.vocab", vocab_size)

Duplicate: k
Duplicate: n
Duplicate: macd
Duplicate: a
Duplicate: e
Duplicate: b
Duplicate: wind
Duplicate: app
Duplicate: token
Duplicate: c
Duplicate: p2p
Duplicate: kdj
Duplicate: bp
Duplicate: okex
Duplicate: app
Duplicate: fcoin
Duplicate: l
Duplicate: i
Duplicate: wind
Duplicate: x
Duplicate: st
Duplicate: 5g
Duplicate: t
Duplicate: eos
Duplicate: v
Duplicate: d
Duplicate: shibor
Duplicate: btc
Duplicate: end
Duplicate: boll
Duplicate: end
Duplicate: ipo
Duplicate: pct
Duplicate: pos
Duplicate: pp
Duplicate: w
Duplicate: coindesk
Duplicate: eth
Duplicate: shibor
Duplicate: hibor
Duplicate: libor
Duplicate: ps
Duplicate: t0
Duplicate: xx
Duplicate: okex
Duplicate: ok
Duplicate: sto
Duplicate: token
Duplicate: choice
Duplicate: okcoin
Duplicate: 24h
Duplicate: visa
Duplicate: bitcoin
Duplicate: usdt
Duplicate: coinmarketcap
Duplicate: msci
Duplicate: g
Duplicate: coindesk
Duplicate: m
Duplicate: vs
Duplicate: one
Duplicate: j
Duplicate: p
Duplicate: ico
Duplicate: mt
Duplicate: qq


In [ ]:
def article2ids(article_words, vocab):
    """Map the article words to their ids. Also return a list of OOVs in the article.

    Args:
      article_words: list of words (strings)
      vocab: Vocabulary object

    Returns:
      ids:
        A list of word ids (integers); OOVs are represented by their temporary article OOV number. If the vocabulary size is 50k and the article has 3 OOVs, then these temporary OOV numbers will be 50000, 50001, 50002.
      oovs:
        A list of the OOV words in the article (strings), in the order corresponding to their temporary article OOV numbers."""
    ids = []
    oovs = []
    for w in article_words:
        i = vocab.word2id(w)
        if i == unk:  # If w is OOV
            if w not in oovs:  # Add to list of OOVs
                oovs.append(w)
            oov_num = oovs.index(w)  # This is 0 for the first article OOV, 1 for the second article OOV...
            ids.append(vocab.size() + oov_num)  # This is e.g. 50000 for the first article OOV, 50001 for the second...
        else:
            ids.append(i)
    return ids, oovs 

In [ ]:
def title2ids(title_words, vocab, article_oovs):
    """Map the title words to their ids. In-article OOVs are mapped to their temporary OOV numbers.

    Args:
      abstract_words: list of words (strings)
      vocab: Vocabulary object
      article_oovs: list of in-article OOV words (strings), in the order corresponding to their temporary article OOV numbers

    Returns:
      ids: List of ids (integers). In-article OOV words are mapped to their temporary OOV numbers. Out-of-article OOV words are mapped to the UNK token id."""
    ids = []
    for w in title_words:
        i = vocab.word2id(w)
        if i == unk:  # If w is an OOV word
            if w in article_oovs:  # If w is an in-article OOV
                vocab_idx = vocab.size() + article_oovs.index(w)  # Map to its temporary article OOV number
                ids.append(vocab_idx)
            else:  # If w is an out-of-article OOV
                ids.append(unk)  # Map to the UNK token id
        else:
            ids.append(i)
    return ids

In [ ]:
def rpadd(x, maxlen=maxlenh, eos=eos, prefix=None):
    
    if prefix != None:
        x = [prefix] + x
    n = len(x)
    if n > maxlen - 1:
        x = x[:maxlen - 1]
        n = maxlen - 1
    res = x + [eos] + [emp] * (maxlen - 1 - n)
    
    assert len(res) == maxlen
    
    return res

In [ ]:
def make_batch(titles_batch, article_batch):
    
    example_list = []
    for i in range(len(titles_batch)):
        article_ids, article_oovs = article2ids(article_batch[i], vocab)
        title_ids = title2ids(titles_batch[i], vocab, article_oovs)
        example_list.append((article_ids, title_ids, article_oovs))
#         print(article_oovs)
    max_art_oovs = max([len(ex[2]) for ex in example_list])
    art_oovs = [ex[2] for ex in example_list]      
    encoder_word_inputs = []
    for i, ex in enumerate(example_list):
        encoder_word_inputs.append(ex[0])
    decoder_inputs = [ex[1] for ex in example_list]
    
    return max_art_oovs,  encoder_word_inputs, decoder_inputs

In [ ]:
# building model

feed_batch_size = tf.placeholder(tf.int32, [], name="feed_batch_size")
encoder_word_inputs = tf.placeholder(tf.int32, shape=[None, maxlend], name="encoder_word_inputs")
encoder_pos_inputs = tf.placeholder(tf.int32, shape=[None, maxlend], name="encoder_pos_inputs")
decoder_targets = tf.placeholder(tf.int32, shape=[None, maxlenh], name="decoder_targets")
decoder_inputs = tf.placeholder(tf.int32, shape=[None, maxlenh], name="decoder_inputs")
max_art_oovs = tf.placeholder(tf.int32, [], name='max_art_oovs')


writer = tf.summary.FileWriter(tensorboard_log_path, graph=tf.get_default_graph())


# user_embeddings = tf.placeholder(tf.float32, shape=[vocab_size, embedding_size], name="user_embeddings")
word_embeddings = tf.get_variable(initializer=tf.truncated_normal(shape=[vocab_size, embedding_size], stddev=0.1), name="word_embeddings")
pos_embeddings = tf.Variable(tf.random_uniform([maxlend+3, embedding_size], -1,0, 1.0), name="pos_embeddings")

# writer = tf.summary.FileWriter(tensorboard_log_path, graph=tf.get_default_graph())


encoder_word_inputs_embedded = tf.nn.embedding_lookup(word_embeddings, encoder_word_inputs)
enocder_pos_inputs_embedded = tf.nn.embedding_lookup(pos_embeddings, encoder_pos_inputs)
encoder_inputs_embedded = encoder_word_inputs_embedded + enocder_pos_inputs_embedded
embed_expanded = tf.expand_dims(encoder_inputs_embedded, -1)

def CNN_C_encoder(embed_expanded):
        
    cnn_c_inputs = embed_expanded
    for i, filter_size in enumerate(filter_sizes):
        with tf.variable_scope("conv-c-%s" % i):
            filter_shape = [filter_size, embedding_size, 1, num_filters]
            W = tf.get_variable(initializer=tf.truncated_normal(filter_shape, stddev=0.1), name="W")
            b = tf.get_variable(initializer=tf.constant(0.1, shape=[num_filters]), name="b")
            conv = tf.nn.conv2d(embed_expanded, W, strides=[1,1,1,1], padding="SAME", name="conv")
            cnn_c_inputs = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
            print(W.name)
        
    h = tf.squeeze(cnn_c_inputs, axis=3)
#     with tf.name_scope("dropout"):
#         h_drop = tf.nn.dropout(h, dropout_keep_prob, name="dropout")
    
    return h

In [ ]:
cnn_c_outputs = CNN_C_encoder(embed_expanded)
print(cnn_c_outputs.shape)

conv-c-0/W:0
conv-c-1/W:0
conv-c-2/W:0
conv-c-3/W:0
conv-c-4/W:0
(?, 200, 100)


In [ ]:
def CNN_A_encoder(embed_expanded):
    
    cnn_a_inputs = embed_expanded
    for i, filter_size in enumerate(filter_sizes):
        with tf.variable_scope("conv-a-%s" % i):
            filter_shape = [filter_size, embedding_size, 1, num_filters]

            W = tf.get_variable(initializer=tf.truncated_normal(filter_shape, stddev=0.1), name="W")
            b = tf.get_variable(initializer=tf.constant(0.1, shape=[num_filters]), name="b")
            conv = tf.nn.conv2d(embed_expanded, W, strides=[1,1,1,1], padding="SAME", name="conv")
            cnn_a_inputs = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
            
            print(W.name)
        
    h = tf.squeeze(cnn_a_inputs, axis=3)
    
#     with tf.name_scope("dropout"):
#         h_drop = tf.nn.dropout(h, dropout_keep_prob, name="dropout")
    
    return h

In [ ]:
cnn_a_outouts = CNN_A_encoder(embed_expanded)
print(cnn_a_outouts.shape)

conv-a-0/W:0
conv-a-1/W:0
conv-a-2/W:0
conv-a-3/W:0
conv-a-4/W:0
(?, 200, 100)


In [ ]:


def LSTM_decoder(cnn_a_outouts, cnn_c_outputs, decoder_inputs):
    
    decoder_inputs_embedded = tf.nn.embedding_lookup(word_embeddings, decoder_inputs)
    lstm = tf.contrib.rnn.BasicLSTMCell(memory_dim)
    decoder_outputs = []
    with tf.variable_scope("attetion"):
        state = lstm.zero_state(feed_batch_size, tf.float32)
#         cell_init = tf.placeholder(tf.float32, shape=[None, memory_dim], name="init_cell")
#         hidden_init = tf.placeholder(tf.float32, shape=[None, memory_dim], name="init_hidden")
#         state = rnn.LSTMStateTuple(cell_init, hid_init)
        W = tf.get_variable(initializer=tf.truncated_normal([memory_dim, embedding_size], stddev=0.1), name="W")
        b = tf.get_variable(initializer=tf.constant(0.1, shape=[embedding_size]), name="b")
        print(W.name)
    
    decoder_outputs = []
    p_gens = []
    attens = []

#     cns = []
    with tf.variable_scope("RNN"):
        for i in range(maxlenh): 
            d = tf.expand_dims(tf.matmul(state[1], W) + b + decoder_inputs_embedded[:,i,:], -1)
            alpha = tf.expand_dims(tf.nn.softmax(tf.squeeze(tf.matmul(cnn_a_outouts, d), axis=-1), dim=1), axis=-1)
            context = tf.reduce_sum(tf.multiply(alpha, cnn_c_outputs), axis=1)
            current_input = tf.concat([context, decoder_inputs_embedded[:,i,:]], axis=1)

            if i > 0:
                tf.get_variable_scope().reuse_variables() 

            decoder_output, state = lstm(current_input, state)
            decoder_outputs.append(decoder_output)
            
            attens.append(alpha)
            with tf.variable_scope("calculate_pgen"):
                p_gen = tf.contrib.layers.fully_connected(tf.concat([current_input, state.c, state.h], axis=1), num_outputs=1, 
                                                          activation_fn=tf.sigmoid)
                p_gens.append(p_gen)
        
#             cns.append(cnn_c_outputs)
    return decoder_outputs, p_gens, attens

In [ ]:
decoder_outputs, p_gens, attens =  LSTM_decoder(cnn_a_outouts, cnn_c_outputs, decoder_inputs)

attetion/W:0


In [ ]:
attens[0].shape

TensorShape([Dimension(None), Dimension(200), Dimension(1)])

In [ ]:
def calc_final_dist(v_size, _max_art_oovs, _enc_batch_extend_vocab, p_gen, vocab_dist, attn_dist):
    """Calculate the final distribution, for the pointer-generator model
    Args:
      vocab_dists: The vocabulary distributions. List length max_dec_steps of (batch_size, vsize) arrays. The words are in the order they appear in the vocabulary file.
      attn_dists: The attention distributions. List length max_dec_steps of (batch_size, max_enc_steps) arrays

    Returns:
      final_dists: The final distributions. List length max_dec_steps of (batch_size, extended_vsize) arrays.
    """
    with tf.variable_scope('final_distribution'):
        # Multiply vocab dists by p_gen and attention dists by (1-p_gen)
        vocab_dist = p_gen * vocab_dist
        attn_dist = (1 - p_gen) * attn_dist

        # Concatenate some zeros to each vocabulary dist, to hold the probabilities for in-article OOV words
        extended_vsize = v_size + _max_art_oovs  # the maximum (over the batch) size of the extended vocabulary
        extra_zeros = tf.zeros((feed_batch_size, _max_art_oovs))
        vocab_dists_extended = tf.concat(axis=1, values=[vocab_dist,
                                                         extra_zeros])  # list length max_dec_steps of shape (batch_size, extended_vsize)

        # Project the values in the attention distributions onto the appropriate entries in the final distributions
        # This means that if a_i = 0.1 and the ith encoder word is w, and w has index 500 in the vocabulary, then we add 0.1 onto the 500th entry of the final distribution
        # This is done for each decoder timestep.
        # This is fiddly; we use tf.scatter_nd to do the projection
        batch_nums = tf.range(0, limit=feed_batch_size)  # shape (batch_size)
        batch_nums = tf.expand_dims(batch_nums, 1)  # shape (batch_size, 1)
        attn_len = tf.shape(_enc_batch_extend_vocab)[1]  # number of states we attend over
        batch_nums = tf.tile(batch_nums, [1, attn_len])  # shape (batch_size, attn_len)
        indices = tf.stack((batch_nums, _enc_batch_extend_vocab), axis=2)  # shape (batch_size, enc_t, 2)
        shape = [feed_batch_size, extended_vsize]
        attn_dists_projected = tf.scatter_nd(indices, attn_dist,
                                             shape)  # list length max_dec_steps (batch_size, extended_vsize)

        # Add the vocab distributions and the copy distributions together to get the final distributions
        # final_dists is a list length max_dec_steps; each entry is a tensor shape (batch_size, extended_vsize) giving the final distribution for that decoder timestep
        # Note that for decoder timesteps and examples corresponding to a [PAD] token, this is junk - ignore.
        final_dist = vocab_dists_extended + attn_dists_projected
        final_dist += 1e-15  # for cases where we have zero in the final dist, especially for oov words
        dist_sums = tf.reduce_sum(final_dist, axis=1)
        final_dist = final_dist / tf.reshape(dist_sums, [-1, 1])  # re-normalize

    return final_dist

In [ ]:

def feed_forward(decoder_outputs, p_gens, attens):
    final_dists = []
    with tf.variable_scope("FNN"):
        w = tf.get_variable(name="W", initializer=tf.truncated_normal(shape=[memory_dim, vocab_size], stddev=0.1))
        b = tf.get_variable(name="b", initializer=tf.constant(0.1, shape=[vocab_size]))
    with tf.variable_scope("feed_forward") as scope:   
        for i in range(maxlent):
            decoder_output, p_gen, atten = decoder_outputs[i], p_gens[i], tf.squeeze(attens[i], axis=-1)
            
            decoder_logit = tf.nn.softmax(tf.matmul(decoder_output, w) + b)
            final_dist = calc_final_dist(vocab_size, max_art_oovs, encoder_word_inputs, p_gen, decoder_logit, atten)
            final_dists.append(final_dist)
    return final_dists

In [ ]:
final_dists = feed_forward(decoder_outputs, p_gens, attens)
stepwise_cross_entropys = []
decoder_targets_list = tf.unstack(decoder_targets, axis=1)
for i in range(maxlent):
    
    label = tf.one_hot(decoder_targets_list[i], depth=max_art_oovs+vocab_size, dtype=tf.float32)
    final_dist = final_dists[i]
    stepwise_cross_entropys.append(tf.reduce_sum(-label * tf.log(final_dist))/tf.cast(feed_batch_size, dtype=tf.float32))

In [ ]:
loss = tf.reduce_mean(tf.stack(stepwise_cross_entropys, axis=0))
decoder_prediction = tf.argmax(tf.stack(final_dists, axis=0), 2, name="decoder_prediction")
train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss, name="op_adam_minize")
saver = tf.train.Saver()

In [ ]:
with tf.variable_scope("summary"):
    tf.summary.scalar("loss", loss)
    tf.summary.histogram("p_gens", tf.stack(p_gens, axis=0))
    for var in tf.trainable_variables():
        tf.summary.histogram(var.name, var)
    for i, ce in enumerate(stepwise_cross_entropys):
        tf.summary.scalar("stepwise_cross_entropy", ce)
    for final_dist in final_dists:
        tf.summary.histogram("final_dist", final_dist)
summary_op = tf.summary.merge_all()

INFO:tensorflow:Summary name word_embeddings:0 is illegal; using word_embeddings_0 instead.
INFO:tensorflow:Summary name pos_embeddings:0 is illegal; using pos_embeddings_0 instead.
INFO:tensorflow:Summary name conv-c-0/W:0 is illegal; using conv-c-0/W_0 instead.
INFO:tensorflow:Summary name conv-c-0/b:0 is illegal; using conv-c-0/b_0 instead.
INFO:tensorflow:Summary name conv-c-1/W:0 is illegal; using conv-c-1/W_0 instead.
INFO:tensorflow:Summary name conv-c-1/b:0 is illegal; using conv-c-1/b_0 instead.
INFO:tensorflow:Summary name conv-c-2/W:0 is illegal; using conv-c-2/W_0 instead.
INFO:tensorflow:Summary name conv-c-2/b:0 is illegal; using conv-c-2/b_0 instead.
INFO:tensorflow:Summary name conv-c-3/W:0 is illegal; using conv-c-3/W_0 instead.
INFO:tensorflow:Summary name conv-c-3/b:0 is illegal; using conv-c-3/b_0 instead.
INFO:tensorflow:Summary name conv-c-4/W:0 is illegal; using conv-c-4/W_0 instead.
INFO:tensorflow:Summary name conv-c-4/b:0 is illegal; using conv-c-4/b_0 instead

In [ ]:
def ids2sent(x):
    sent = []
    for idx in x:
#         if idx == emp:
#             continue
        if idx == eos:
            break
        sent.append(idx2word[idx])
    
    return " ".join(sent)
        

In [ ]:
def predict(X_test, Y_test, sess, log_dir):
    
    batch_size = 1
    
    rouge_mea = {
        'rouge_1/f_score': 0.0,
        'rouge_1/r_score': 0.0,
        'rouge_1/p_score': 0.0,
        'rouge_2/f_score': 0.0,
        'rouge_2/r_score': 0.0,
        'rouge_2/p_score': 0.0,
        'rouge_l/f_score': 0.0,
        'rouge_l/r_score': 0.0,
        'rouge_l/p_score': 0.0
    }
    outf = open(log_dir, 'w', encoding='utf-8')
    j = 0
    while j < len(X_test):
        
        titles_batch, abstracts_batch = title_train[j:j+batch_size], abstract_train[j:j+batch_size]
        max_art_oovs_, X_train, Y_train = make_batch(titles_batch, abstracts_batch)
        encoder_word_inputs_ = list(map(lambda x: rpadd(x, maxlend), X_train))
        encoder_pos_inputs_ = list(map(lambda x:rpadd(list(range(3,len(x)+3)), maxlend), X_train))
        decoder_inputs_ = list(map(lambda x: rpadd(x, maxlenh, prefix=beg), Y_train))
        decoder_targets_ = list(map(lambda x: x[1:]+[emp], decoder_inputs_))
        j = j + batch_size
        
      
        test_x = [[] for _ in range(len(encoder_word_inputs_))]
        for l in range(maxlenh):
            new_decoder_input_ = list(map(lambda x: rpadd(x, maxlenh, prefix=beg), test_x))

            decoder_prediction_= sess.run(decoder_prediction,
                                          feed_dict={
                                              encoder_word_inputs: encoder_word_inputs_,
                                              encoder_pos_inputs: encoder_pos_inputs_,
                                              decoder_inputs: new_decoder_input_,
                                              feed_batch_size: 1,
                                              max_art_oovs: max_art_oovs_
                                          })
            for b in range(len(encoder_word_inputs_)):
                next_word_idx = decoder_prediction_[b][l]
                test_x[b].append(next_word_idx)
                        
                
        # compute 'rouge' measure in the batch
        hypotheses = [ids2sent(single) for single in test_x]
        references = [ids2sent(single) for single in decoder_inputs_]
        
        rouge_batch = rouge(hypotheses, references)
        
        for key in rouge_mea.keys():
            rouge_mea[key] += rouge_batch[key]

        # show batch result in file 
        for b in range(len(encoder_word_inputs_)):
            
            outf.write("{0}\t{1}\t{2}\n".format(ids2sent(test_x[b]), ids2sent(decoder_inputs_[b])
                                                , ids2sent(encoder_word_inputs_[b])))
            
    # compute rouge measue in the test
    outf.write("------rouge measure------")
    for key in rouge_mea.keys():
        rouge_mea[key] = rouge_mea[key] / (len(X_test)//batch_size)
        outf.write("{0}: {1}\n".format(key, rouge_mea[key]))

    
    outf.close()

In [ ]:
import numpy as np
sess = tf.Session()
sess.run(tf.global_variables_initializer())
graph = tf.get_default_graph()

if restore:
    saver.restore(sess, tf.train.latest_checkpoint("./ckpt/"))
if inference:
    predict(X_test, Y_test, sess, inference_fn)

if train:
    print("start trainging...")  
    for i in range(epocs):
        j = 0
        while j < len(title_train):
            
            titles_batch, abstracts_batch = title_train[j:j+batch_size], abstract_train[j:j+batch_size]
            max_art_oovs_, X_train, Y_train = make_batch(titles_batch, abstracts_batch)
            encoder_word_inputs_ = list(map(lambda x: rpadd(x, maxlend), X_train))
            encoder_pos_inputs_ = list(map(lambda x:rpadd(list(range(3,len(x)+3)), maxlend), X_train))
            decoder_inputs_ = list(map(lambda x: rpadd(x, maxlenh, prefix=beg), Y_train))
            decoder_targets_ = list(map(lambda x: x[1:]+[emp], decoder_inputs_))
            j = j + batch_size
            summary, _, loss_, decoder_prediction_ = sess.run([summary_op, train_op, loss, decoder_prediction],
                                                             feed_dict={
                                                                 encoder_word_inputs: encoder_word_inputs_,
                                                                 encoder_pos_inputs: encoder_pos_inputs_,
                                                                 decoder_inputs: decoder_inputs_,
                                                                 decoder_targets: decoder_targets_,
                                                                 feed_batch_size: len(encoder_word_inputs_),
                                                                 max_art_oovs: max_art_oovs_
                                                             })
            writer.add_summary(summary, i*len(X_train) + j)
            
            # debug
            A, B = sess.run([final_dists[0], tf.one_hot(decoder_targets_list[0], depth=max_art_oovs+vocab_size, dtype=tf.float32)], 
                           feed_dict={
                                 encoder_word_inputs: encoder_word_inputs_,
                                 encoder_pos_inputs: encoder_pos_inputs_,
                                 decoder_inputs: decoder_inputs_,
                                 decoder_targets: decoder_targets_,
                                 feed_batch_size: len(encoder_word_inputs_),
                                 max_art_oovs: max_art_oovs_
                             })
    
            if j % (batch_size * 30) == 0:
                logger.info("Runing in epoc {0} batch {1} with loss {2}".format(i, j//batch_size, loss_))


        if i % save_epoc_step == 0:
            saver.save(sess, "./ckpt/{0}".format(model_dump_dir), global_step=i)
    print("finish trainging.") 



start trainging...


In [ ]:
j = 0
titles_batch, abstracts_batch = title_train[j:j+batch_size], abstract_train[j:j+batch_size]
max_art_oovs_,  X_train, Y_train = make_batch(titles_batch, abstracts_batch)
decoder_inputs_ = list(map(lambda x: rpadd(x, maxlenh, prefix=beg), Y_train))
decoder_targets_ = list(map(lambda x: x[1:]+[emp], decoder_inputs_))

In [ ]:
decoder_targets = tf.constant(value=decoder_inputs_)
decoder_targets_list = tf.unstack(decoder_targets, axis=1)

In [ ]:
max_art_oovs = tf.constant(value=100)

In [ ]:
label = tf.one_hot(decoder_targets_list[0], depth=max_art_oovs+vocab_size, dtype=tf.float32)

In [ ]:
label.shape

In [ ]:
# cite: https://arxiv.org/pdf/1702.01806.pdf
# title: Beam Search Strategies for Neural Machine Translation
# beam search with pruning (naive, version)
# prune hypo parameter

rp = 0.2
ap = 2.5
rpl = 0.3
mc = 2
beam_size = 30000

def prune(beam_tmp, decoder_probs_l):
    
#     print([[idx2word[idx] for idx in seq] for score, seq in beam_tmp])
    
#     max_score, max_seq = beam_tmp[0]
#     print("max_score: {0}, max_seq: {1}".format(max_score, max_seq))
#     beam_tmp = list(filter(lambda x: x[0] > rp * max_score, beam_tmp))
#     beam_tmp = list(filter(lambda x: x[0] > max_score - ap, beam_tmp))
#     beam_tmp = list(filter(lambda x: decoder_probs_l[x[1][-1]] > rpl * decoder_probs_l[max_seq[-1]], beam_tmp))
    
#     beam_res = []
#     cnt = {}
#     for prob, seq in beam_tmp:
#         key = "_".join([str(s) for s in seq])
#         if key not in cnt:
#             cnt[key] = 0
#         if cnt[key] < mc:
#             beam_res.append((prob, seq))
#             cnt[key] += 1
            
#     print([[idx2word[idx] for idx in seq] for score, seq in beam_res])

    
    
    return beam_res

def beam_search_with_pruning(X_one, Y_one, sess, beam_size):
    
    encoder_word_inputs_ = [rpadd(X_one, maxlend)]
    encoder_pos_inputs_ = [rpadd(list(range(3,len(X_one)+3)), maxlend)]
    
    beam_list = [(1.0, [beg])] 
    final_list = []
    
    for l in range(maxlenh):

        candidate = {}
        for prob_seq, seq in beam_list:
            
            
            new_decoder_input_ = [rpadd(seq, maxlenh, prefix=None)]
   
            decoder_logits_ = sess.run(tf.log(tf.stack()),
                                          feed_dict={
                                              encoder_word_inputs: encoder_word_inputs_,
                                              encoder_pos_inputs: encoder_pos_inputs_,
                                              decoder_inputs: new_decoder_input_,
                                              feed_batch_size: 1,
                                              max_art_oovs: max_art_oovs_    
                                          })
            decoder_probs_l = decoder_logits_[0][l]
            probs_top_ids = np.argsort(decoder_probs_l)[-beam_size:]
#             print([idx2word[idx] for idx in probs_top_ids])
            
#             print("------------------------------")
            
            for v in probs_top_ids:
                if v == unk or v == emp: continue
                pt = prob_seq + decoder_probs_l[v]
                candidate[pt] = seq + [v]
                
       
 
        res_sort = sorted(list(candidate.items()), key=lambda x: x[0], reverse=True)   
        
        beam_tmp = res_sort[:beam_size]
        
        # beam_tmp = prune(beam_tmp, decoder_probs_l)  
                
        beam_list.clear()
        
        for prob, seq in beam_tmp:
            if seq[-1] == eos:
                final_list.append((prob, seq))
                beam_size -= 1
            else:
                beam_list.append((prob, seq))
            
        if len(beam_list) == 0 or beam_size == 0:
            break
           
    return [(ids2sent(item[1]), item[0]) for item in final_list]

In [ ]:
t = beam_search_with_pruning(X_test[3], Y_test[3], sess, 4)

In [ ]:
predict_with_beam(beam_size=4, filename="./log/produce.beam.samples", sess=sess, X_test=X_test, Y_test=Y_test)

In [ ]:
ids2sent(X_test[3])

In [ ]:
ids2sent(Y_test[3])

In [ ]:
def predict_with_beam(X_test, Y_test, sess, beam_size, filename):
    rouge_mea = {
        'rouge_1/f_score': 0.0,
        'rouge_1/r_score': 0.0,
        'rouge_1/p_score': 0.0,
        'rouge_2/f_score': 0.0,
        'rouge_2/r_score': 0.0,
        'rouge_2/p_score': 0.0,
        'rouge_l/f_score': 0.0,
        'rouge_l/r_score': 0.0,
        'rouge_l/p_score': 0.0
    }
    
    hypotheses = []
    references = []
    out = open(filename, "w", encoding="utf-8")
    for x, y in zip(X_test, Y_test):
        res = beam_search_with_pruning(x, y, sess, beam_size)[0][0]
        out.write("{0}\t{1}\t{2}\n".format(res, ids2sent(y), ids2sent(x)))
        out.flush()
    
 
        hypotheses.append(ids2sent(x))
        references.append(res)
        
    rouge_mea = rouge(hypotheses, references)

    # compute rouge measue in the test
    outf.write("------rouge measure------")
    for key in rouge_mea.keys():
        rouge_mea[key] = rouge_mea[key] / (len(X_test)//batch_size)
        outf.write("{0}: {1}\n".format(key, rouge_mea[key]))

    
    outf.close()


In [ ]:
predict_online(X_test, Y_test, sess, test_content_words)

In [ ]:
find_proper(t, test_content_words[3], 5)

In [ ]:
" ".join([w for w in test_content_words[0].split(" ") if w not in stopwords])

In [ ]:
t

In [ ]:
real = []
for word in can.split(" ")[1:]:
    if word in test_content_ner2words[1]:
        real.append(test_content_ner2words[1][word][0])
    else:
        real.append(word)
print(real)

In [ ]:
test_content_ner2words[3]

In [ ]:
for a in t:
    r = rouge([a[0]], [test_content_words[3]])
    print(a[0])
    print(test_content_words[3])
    print(r)

In [ ]:
import scipy
1 - scipy.spatial.distance.cosine(embed[word2idx["你"]],embed[word2idx["我"]])

In [ ]:
# 测试词向量

def neighbor(target_word, k):
    
    w2sim = {}
    for word in word2idx.keys():
        sim = 1 - scipy.spatial.distance.cosine(embed[word2idx[target_word]],embed[word2idx[word]]) 
        w2sim[word] = sim
    
    return sorted(w2sim.items(), key=lambda x:x[1], reverse=True)[:k]

In [ ]:
ns = neighbor("外汇", 20)
for n in ns:
    print(n)